In [ ]:
import argparse
import json
import os

import torch

from csnet.utils.model import get_model
from csnet.utils.predict import predict

In [ ]:
input_dir = 'data/semantic_3D/test/img'
output_dir = 'predictions/test'
model_path = 'model_test/dry-bush-38/best_model.pth'

batch_size = 4

### Setup and load model 

In [ ]:
with open(os.path.join(os.path.dirname(model_path), 'config.json')) as f:
    config = json.load(f)
config = argparse.Namespace(**config)
config

In [ ]:
net = get_model(config)
net.load_state_dict(torch.load(model_path))

### Predict

In [ ]:
image, predicted = predict(input_dir, output_dir, net, roi_size=config.roi_size, return_last=True, batch_size=batch_size)

In [ ]:
plot_projections([image, predicted], panel_size=6)